### 1. Titre du projet
Evaluation  des zones inondables dans le bassin du fleuve Gambie à partir d'un scénario de crue localisée - Cas de la station de Kédougou (Juillet 2024)

### 2. Contexte et objectif
Ce projet vise à localiser les zones critiques en aval de la station de Kédougou en croisant le MNT, l'occupation du sol et les données GHSL

### 3. Données utilisées
- MNT du bassin versant du fleuve Gambie
- Landcover ESA WorldCover 
- Données GHSL (population et surface bâtie)
- Données de crue ()
  * Nivau d'alerte (Juillet 2024):
  * Niveau observé :
  * Ecart modélisé

### 4. Technologies utilisées
- PCRaser: moteur principal de simulation pour la modélisation hydrologique, nottamment les fonctions accucapacityflux, accucapacitystate, spreadzone, ldd, etc.
- GDAL: convertion et manipulation de fichier raster (GeoTiff vers .map, reprojection, extraction de métadonnées).
- Numpy: manipulation efficace de matrice (ex. pour les conversions intermédiares ou calculs personnalisés).
- Matplotlib: visualisation des résultats(cartes des zones inondables, évolution temporelle).
- Jupyter notebook: environnement interactif pour tester, visualiser et documenter les simulations

### 5. Choix d'un modèle dynamique
Pour simuler le processus d'inondation sur le bassin Gambie, nous avons choisi d'utiliser un **modèle dynamique PCRaster** basé sur la classe `DynamicModel`.
#### Pourquoi un modèle dynamique?
- **Evolution temporelle** : Les phénomènes hydrologiques(écoulement, inondation, ruissellement) sont dépendants du temps. Le modèle dynamique permet d'intégrer cette évolution pas à pas.
- **Simulation réaliste** : Il permet d'observer l'impact cumulé des précipitation, de la topographie, de l'occupation du sol, etc sur la propagation de l'eau.
- **Flexibilité** : Grâce aux itérations temporelles, il est possible d'intégrer des paramètres variables dans le temps(ex. précipitations journalières, gestion des barrages, etc.).
Ce modèle est donc adapté pour reproduire et analyser le comportement spatial et  temporel des inondations dans le bassin.

### 6. Séparation des traitement statiques et dynamiques
Dans un souci de performation et de modularité, certains traitements geospataux lourds ou invariant ont été externalisés de la classe `DynamicModel`. L'objectif est de les exécuter une seule fois en amont, puis sauvegarder les résultats pour les réutilisés dans la simulation dynamiques

### 7. Traitements préalables effectués hors de `DynamicModel`

#### a. Importation des bibliothèques python

In [1]:
import pcraster as pcr
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from osgeo import gdal, osr, gdalconst
from tqdm import tqdm # pour une barre de progression
from pyproj import Transformer
from riverrem.REMMaker import REMMaker
import math
gdal.UseExceptions()

In [3]:
#rem_maker = REMMaker(dem='Data/basincover.tif', out_dir='./Data')
#rem_maker.make_rem()
#rem_maker.make_rem_viz(cmap='topo')

In [2]:
#dem = gdal.Open('Data/basin_dem.tif') # le Modèle Numérique Terrestre
#landcover = gdal.Open('Data/basincover.tif') # les données sur l'occupation du sol
#ghsl_built = gdal.Open('Data/basin_built.tif') # les données GHSL de la distribution des surfaces urbanisées (bâties), exprimées en mètres carrés 2020
#ghsl_pop = gdal.Open('Data/basin_pop.tif') # les données GHSL de la distribution spatiale de la population en 2020

#### b. Imporation des données et affichage des proprietés de nos dataset

In [4]:
# Fonction pour l'affichage des proprietés des datasets
def gdal_properties(input_path): # Affichier les proprietés pour chaque fichier
    if not os.path.exists(input_path):
        raise ValueError(f"Fichier introuvable")
    raster = gdal.Open(input_path)
    path = raster.GetDescription()
    width = raster.RasterXSize
    height = raster.RasterYSize
    projection = raster.GetProjection()
    geotransform = raster.GetGeoTransform()
    metadata = raster.GetMetadata()
    numband = raster.RasterCount # chacune de nos rasters ne renferme qu'une bande
    
    band = raster.GetRasterBand(1)
    data_type = band.DataType
    data_type_name = gdal.GetDataTypeName(data_type)
    print(f"Le type de données de la bande 1: {data_type_name}")
    driver = raster.GetDriver().ShortName
    print("#############################################")
    print('Propriétés')
    print(f"{path} a {width} x {height} comme dimension")
    print(f"Driver: {driver}")
    print(f"Nombre de bande: {numband}")
    print(f"Les metadonnées: {metadata}")

    srs = osr.SpatialReference() 
    srs.ImportFromWkt(projection)
    epsg_code = srs.GetAuthorityCode(None)
    unit_name = "Degree" if epsg_code == "4326" else srs.GetLinearUnitsName()
    unit_value = srs.GetLinearUnits()
    print(f"Unité de la projection: {unit_name} {unit_value}")

    if geotransform:
        
        print(f"Origine: ({geotransform[0]}, {geotransform[3]})")
        print(f"Size: ({geotransform[1]} {unit_name}, {geotransform[5]} {unit_name})")
    layerBand = raster.GetRasterBand(1)
    min_value, max_value = layerBand.ComputeRasterMinMax()
    print(f"Valeur minimale {min_value}")
    print(f"Valeur maximale {max_value}")
    print("#############################################")
    raster = None

In [4]:
basin_raster_paths = glob.glob('Data/basin*.tif') # Charger les fichiers geotiff
for path in basin_raster_paths:
    
    gdal_properties(path)
    print('\n')

Le type de données de la bande 1: UInt16
#############################################
Propriétés
Data/basin_built.tif a 20060 x 17852 comme dimension
Driver: GTiff
Nombre de bande: 1
Les metadonnées: {'AREA_OR_POINT': 'Area'}
Unité de la projection: m 1.0
Origine: (-1721000.0, 2052780.0)
Size: (30.0 m, -30.0 m)
Valeur minimale 0.0
Valeur maximale 6599.0
#############################################


Le type de données de la bande 1: Int16
#############################################
Propriétés
Data/basin_dem.tif a 18137 x 12359 comme dimension
Driver: GTiff
Nombre de bande: 1
Les metadonnées: {'AREA_OR_POINT': 'Area'}
Unité de la projection: Degree 1.0
Origine: (-16.15022041715082, 14.68830289986267)
Size: (0.0002694945852358564 Degree, -0.0002694945852358564 Degree)
Valeur minimale -29.0
Valeur maximale 1530.0
#############################################


Le type de données de la bande 1: Float64
#############################################
Propriétés
Data/basin_pop.tif a 15897 

#### c. Alignement des rasters
Pour une cohérence spatiales, une interopérabilité entre les données et une optimisation des calculs, on va aligner nos rasters sur une même grille, avec:
- une projection commune(EPSG:32628 - UTM Zone 28N)
- Une résolution fixe (100m). Le choix de cette résolution repose sur un compromis entre la capacité de traitement disponible et une précision géographique. Une résolution fine (30m, 10m) offrirait plus de détails mais générerait des rasters de grande taille difficillement manipulables dans une environnement contraint comme mon ordinateur personnel avec une RAM(2Go) un processeur(Intel celeron CPU N3050) limité.
- Une origine spatiale commune bassée sur le MNT Projeté

Etape 1: Reprojection du MNT

In [5]:
def reproject_raster_get_origin(input_raster, target_epsg=32628, resolution=30):
    output_raster = "Data/mnt_utm.tif"
    warp_options = gdal.WarpOptions(
        dstSRS=f"EPSG:{target_epsg}",
        xRes=resolution,
        yRes=resolution,
        resampleAlg='bilinear'
    )
    gdal.Warp(output_raster, input_raster, options=warp_options)

    ds = gdal.Open(output_raster)
    gt = ds.GetGeoTransform()
    origin_x = gt[0]
    origin_y = gt[3]
    width = ds.RasterXSize
    height = ds.RasterYSize
    ds = None
    return origin_x, origin_y, width, height, output_raster


Etape 2: Alignement des autres rasters

In [6]:
def align_raster(input_raster, output_raster, reference_epsg, resolution, origin_x, origin_y, width, height, resampleAlg):
    warp_options = gdal.WarpOptions(
        dstSRS=f"EPSG:{reference_epsg}",
        xRes=resolution,
        yRes=resolution,
        outputBounds=(
            origin_x,
            origin_y - resolution * height,
            origin_x + resolution * width,
            origin_y
        ),
        width=width,
        height=height,
        resampleAlg=resampleAlg
    )
    gdal.Warp(output_raster, input_raster, options=warp_options)
    
    print(f"Raster reprojeté et aligné sauvegardé : {output_raster}")
    print('\n')


Etape 3: Utilisation

In [7]:
# Reprojection du MNT et extraction des métadonnées
mnt_path = 'Data/basin_dem.tif'
origin_x, origin_y, width, height, mnt_reprojected = reproject_raster_get_origin(mnt_path)

# Alignment des autre rasters
input_data = {
    'mnt': {
        'input_path': 'Data/basin_dem.tif',
        'output_path': 'Data/Aligned_rasters/mnt_aligned.tif',  # Donnée continue (altitude)
        'resampleAlg': 'bilinear'
    },
    'landcover': {
        'input_path': 'Data/basincover.tif',
        'output_path': 'Data/Aligned_rasters/landcover_aligned.tif',  # Donnée discret (occupation du sol)
        'resampleAlg': 'near'
    },
    'ghsl_built': {
        'input_path': 'Data/basin_built.tif',
        'output_path': 'Data/Aligned_rasters/ghsl_built_aligned.tif',  # Donnée continue (surface bâtie)
        'resampleAlg': 'bilinear'
    },
    'ghsl_pop': {
        'input_path': 'Data/basin_pop.tif',
        'output_path': 'Data/Aligned_rasters/ghsl_pop_aligned.tif',  # Donnée continue (population)
        'resampleAlg': 'bilinear'
    }
}
for name, infos in tqdm(input_data.items()): # tqdm permet d'aficher la progression
    align_raster(
        input_raster=infos['input_path'],
        output_raster=infos['output_path'],
        reference_epsg=32628,
        resolution=100,
        origin_x=origin_x,
        origin_y=origin_y,
        width=width,
        height=height,
        resampleAlg=infos['resampleAlg']
    )


 25%|███████████▎                                 | 1/4 [00:11<00:33, 11.09s/it]

Raster reprojeté et aligné sauvegardé : Data/Aligned_rasters/mnt_aligned.tif




 50%|██████████████████████▌                      | 2/4 [00:14<00:13,  6.53s/it]

Raster reprojeté et aligné sauvegardé : Data/Aligned_rasters/landcover_aligned.tif




 75%|█████████████████████████████████▊           | 3/4 [00:23<00:07,  7.58s/it]

Raster reprojeté et aligné sauvegardé : Data/Aligned_rasters/ghsl_built_aligned.tif




100%|█████████████████████████████████████████████| 4/4 [01:22<00:00, 20.55s/it]

Raster reprojeté et aligné sauvegardé : Data/Aligned_rasters/ghsl_pop_aligned.tif




#### d. Verification de l'alignement des rasters

In [8]:
aligned_raster_paths = glob.glob('Data/Aligned_rasters/*.tif')
for path in aligned_raster_paths:
    
    gdal_properties(path)
    print('\n')

Le type de données de la bande 1: UInt16
#############################################
Propriétés
Data/Aligned_rasters/ghsl_built_aligned.tif a 17786 x 12389 comme dimension
Driver: GTiff
Nombre de bande: 1
Les metadonnées: {'AREA_OR_POINT': 'Area'}
Unité de la projection: metre 1.0
Origine: (374491.5725851272, 1627184.8412821454)
Size: (100.0 metre, -100.0 metre)
Valeur minimale 0.0
Valeur maximale 6022.0
#############################################


Le type de données de la bande 1: Int16
#############################################
Propriétés
Data/Aligned_rasters/mnt_aligned.tif a 17786 x 12389 comme dimension
Driver: GTiff
Nombre de bande: 1
Les metadonnées: {'AREA_OR_POINT': 'Area'}
Unité de la projection: metre 1.0
Origine: (374491.5725851272, 1627184.8412821454)
Size: (100.0 metre, -100.0 metre)
Valeur minimale -10.0
Valeur maximale 1524.0
#############################################


Le type de données de la bande 1: Byte
#############################################
Propr

#### e. Conversion
les coordonées GPS(longitude, latitude) des stations Hydrometrique doivent être convertis en coordonnées image(ligne, colonne).
La conversion est nécessaire lorsqu'on travaille avec des rasters(comme un MNT, un LDD).
- Station Kégoudou (-12.1833, 12.55)
- Station Mako (-12.35, 12.8667)

In [24]:
#définition d'une fonction de conversion des coordonnées géographiques en indice de pixel ou inversement
def geoToPixel(x, y, geotransform, inverse=False):
    # D'abord on va convertir les coordonnées géographiques en coordonnées projetées
    def gps_to_utm(x, y): #fonction pour convertir les coordonnées géographiques en coordonnées projetées
        transformer_wgs = Transformer.from_crs("EPSG:4326", "EPSG:32628", always_xy=True)
        
        x_proj, y_proj = transformer_wgs.transform(x, y)
        return [x_proj, y_proj]
        
    if inverse:
        #print("Convertir Pixel à Géo")
        xcoord = geotransform[0] + x * geotransform[1]
        ycoord = geotransform[3] + y * geotransform[5]
        
    elif inverse is False:
        x_utm, y_utm = gps_to_utm(x, y)
        #print("Convertir Géo a Pixel")
        xcoord = int((x_utm - geotransform[0]) / geotransform[1])
        ycoord = int((y_utm - geotransform[3]) / geotransform[5])
    else:
        print("le raster n'a pas de trasnformation")
    return xcoord, ycoord
#tester la fonction sur notre raster
# Choisir un GeoTransformation à partir de nos raster
geotransform = gdal.Open('Data/Aligned_rasters/mnt_aligned.tif').GetGeoTransform()
kedougou = geoToPixel(-12.1833, 12.55, geotransform)
mako = geoToPixel(-12.35, 12.8667, geotransform)
print(f"Station Kédougou en Coordonées image {kedougou}")
print(f"Station Mako en Coordonées image {mako}")

Station Kédougou en Coordonées image (4316, 2381)
Station Mako en Coordonées image (4131, 2033)


#### f. Traduction les fichiers GeoTiff déja alignés en fichier pcraster(.map)

In [10]:
# Définition de la fonction de conversion
def convert_to_pcraster(input_filename, output_filename, gdal_type,value_scale):

    #gdal.SetConfigOption("PCRASTER_VALUESCLAE", value_scale)
    gdal.UseExceptions()
    src_ds = gdal.Open(input_filename, gdalconst.GA_ReadOnly)
    
    gdal.Translate(output_filename, src_ds, format="PCRaster", outputType=gdal_type, metadataOptions=value_scale)#creationOptions=[f"VALUESCALE={value_scale}"])
    print(f"Converti : {input_filename} en {output_filename}")
    
    #GDAL Translate
    #dst_ds = gdal.Translate(dst_filename, src_ds, format='PCRaster', outputType=ot, metadataOptions=VS)
    
    # Libérer proprement les objets en mémoire
    dst_ds = None
    src_ds = None
    


In [11]:
# Exécution de de la fonction de conversion des fichiers .tif en .map
input_filenames = {
    'Data/Aligned_rasters/mnt_aligned.tif': {
        'output': 'Data/PCRaster_files/mnt.map',
        'value_scale': 'VS_SCALAR',
        'gdal_type': gdalconst.GDT_Float32
    },
    'Data/Aligned_rasters/landcover_aligned.tif': {
        'output': 'Data/PCRaster_files/landcover.map',
        'value_scale': 'VS_NOMINAL',
        'gdal_type': gdalconst.GDT_Byte
    },
'Data/Aligned_rasters/ghsl_built_aligned.tif': {
        'output': 'Data/PCRaster_files/ghsl_built.map',
        'value_scale': 'VS_SCALAR',
        'gdal_type': gdalconst.GDT_Float32
    },
'Data/Aligned_rasters/ghsl_pop_aligned.tif': {
        'output': 'Data/PCRaster_files/ghsl_pop.map',
        'value_scale': 'VS_SCALAR',
        'gdal_type': gdalconst.GDT_Float32
    }
}
# Lancer les conversations
for filename, infos in input_filenames.items():
    convert_to_pcraster(input_filename=filename,
                       output_filename=infos['output'],
                       value_scale=infos['value_scale'],
                       gdal_type=infos['gdal_type'])


Converti : Data/Aligned_rasters/mnt_aligned.tif en Data/PCRaster_files/mnt.map
Converti : Data/Aligned_rasters/landcover_aligned.tif en Data/PCRaster_files/landcover.map
Converti : Data/Aligned_rasters/ghsl_built_aligned.tif en Data/PCRaster_files/ghsl_built.map
Converti : Data/Aligned_rasters/ghsl_pop_aligned.tif en Data/PCRaster_files/ghsl_pop.map


#### g. Génération de LDD (Local Drain Direction)
Le LDD est une composante fondamentale dans toute modélisation hydrologique. Il représente, pour chaque cellule du MNT, la direction dans laquelle l'eau s'écoule vers une cellule voisine. Cette structure est indispensable pour simuler le ruissellement, l'accumulation ou les inondations. Avant de générer la direction de l'écoulement, nous allons corrigé les dépressions sur le Modèle Numérique Terrain.

In [12]:
# Une carte de clone
pcr.setclone('Data/PCRaster_files/mnt.map')

In [6]:
# Fonction de calcul de direction
def calculate_flow_direction(dem_file):
    dem = pcr.readmap(dem_file)
    #dem_filled = pcr.lddcreatedem(dem, 1e31, 1e31, 1e31, 1e31)
    #pcr.report(dem_filled, 'Data/PCRaster_files/mnt_corrige.map')
    
    flowDirection = pcr.lddcreate(dem, 1e31, 1e31, 1e31, 1e31)
    pcr.report(flowDirection, 'Data/PCRaster_files/ldd_30m.map')

#calculate_flow_direction('Data/PCRaster_files/mnt.map')

#### h. Délimition du fleuve Gambie dans le cadre de la modélisation des inondations:
Dans le cadre de ce projet, l'objectif principal de la délimitation du cours est d'identifier précisément le lit majeur du fleuve Gambie, en excluant les affluents secondaires. Cette délimitation est essentielle pour:
- Localiser les zones suceptibles d'être inondées à proximité immédiate du fleuve
- Simuler l'expension spatiale des inondations depuis le fleuve principal.
- Intégrer les stations de mesure (Kédougou et Mako) dans une analyse cohérente et réaliste

##### Méthode utlisée: ordre de strahler: 
Pour identifier le fleuve principale parmi l'ensemble des réseaux hydrographique, nous allons utilisé la technique basée sur **l'ordre de Strahler** qui permet de:
+ classer les rivières selon leur importance topologique dans un réseau hydrographique
+ Filtrer les affuents secondaires à l'aide d'un seuil élevé
+ Générer une carte du cours d'eau principal à partir d'un LDD (local Drainage Direction)

In [33]:
# Calculer l'ordre de Strahler et extraire le réseau pour chaque ordre
def stream_delineation(flow_direction_map): # Avec comme paramètre, le LDD
    
    orders = pcr.streamorder(flow_direction_map)
    pcr.report(orders, 'Data/Streams/strahler.map') # enregistrer sous forme de carte pcraster les ordres de Strahler
    """order_max = pcr.mapmaximum(orders)
    order_max_value = pcr.cellvalue(order_max, 1, 1)[0]
    for order in range(1, order_max_value+1):
        stream = pcr.ifthen(orders >= order, pcr.scalar(1))
        pcr.report(stream, f'Data/Streams/stream{str(order)}.map')"""
    return orders


In [30]:
# Utlisation de l'ordre de Strahler sur notre réseau de drainage
ldd = pcr.readmap('Data/PCRaster_files/ldd.map')

##### validation 
1. Objectif
L'objectif est de valider si le réseau hydrographique extrait à partir des ordres de Stahler assure une connexion continue entre ls stations de Kédougou et Mako.
2. Méthode
* Filtrage de cours d'eau
Sélectionner uniquement les rivières ayant un ordre de Stahler supèrieur ou égal à un seuil donnée
* Affichage Graphique
Afficher en fond de carte les cours d'eau sélectionner puis placer les stations Kédougou et Mako sur la carte.
* Observation
Vérifier visuellement si les stations Kédougou et Mako tombent dans le masque.
3. Justification de la méthode
* La validation graphique permet d'observer directement la structure du résau hydrographique sans ambiguité
* Elle offre une onfirmation visuelle de la connectivité entre les deux stations, ce qui est plus fiable qu'une validation numérique

In [28]:
#définition d'une fonction de conversion des coordonnées géographiques en indice de pixel ou inversement
def geoToPixel(x, y, geotransform, inverse=False):
    # D'abord on va convertir les coordonnées géographiques en coordonnées projetées
    def gps_to_utm(x, y): #fonction pour convertir les coordonnées géographiques en coordonnées projetées
        transformer_wgs = Transformer.from_crs("EPSG:4326", "EPSG:32628", always_xy=True)
        
        x_proj, y_proj = transformer_wgs.transform(x, y)
        return [x_proj, y_proj]
        
    
    
    if inverse is True: 
        """si inverse est True, x et y sont en coordonnées géographiques, donc on les converties en coordonnées pixel
        après reprojection"""
        #print("Convertir Pixel à Géo")
        xcoord = geotransform[0] + x * geotransform[1]
        ycoord = geotransform[3] + y * geotransform[5]
    
    
    elif inverse is False:
            
        """si inverse est False, x et y sont en coordonnées pixel, donc on les converties en coordonnées projetées
        après reprojection"""
        x_utm, y_utm = gps_to_utm(x, y)
        #print("Convertir Géo a Pixel")
        xcoord = int((x_utm - geotransform[0]) / geotransform[1])
        ycoord = int((y_utm - geotransform[3]) / geotransform[5])
    else:
        print("le raster n'a pas de trasnformation")
    return xcoord, ycoord
#tester la fonction sur notre raster
# Choisir un GeoTransformation à partir de nos raster

#print(f"Station Kédougou en Coordonées image {kedougou}")
#print(f"Station Mako en Coordonées image {mako}")

In [32]:
""" L'exception single resize rencontré lors de l'utilisation de pcr2numpy provient d'une incompatibilité entre la taille
du rastercourant en mémoire dans pcraster et celle du du raster que l'on souhaite convertir. Par conséquent PCRaster
interdit interdit le redimenssionnement automatique pour éviter des incohérence dans les calculs.
Afin de contourner cette contrainte, une fonction personnalisée a été mise en place """
# Fonction personnalisée à la place de pcr2numpy
def pcr_to_Numpy(pcr_map, nodata_value):
    rows = pcr.clone().nrRows()
    cols = pcr.clone().nrCols()
    pcr_array = np.zeros((rows, cols), dtype=np.float32)
    for row in range(0, rows):
        for col in range(0, cols):
            value, mv =  pcr.cellvalue(pcr_map, row + 1, col + 1)
            if mv is False:
                value = nodata_value
            pcr_array[row, col] = value
    #pcr_array = pcr_array.astype(int)
    return pcr_array

In [6]:
#tester la fonction sur notre raster
# Choisir un GeoTransformation à partir de nos raster
geotransform = gdal.Open('Data/Aligned_rasters/mnt_aligned.tif').GetGeoTransform()
kedoudou_x_in_pixel, kedougou_y_in_pixel = geoToPixel(-13.7333, 13.4667, geotransform) # coordonnées en pixel de la station Kédougou
mako_x_in_pixel, mako_y_in_pixel = geoToPixel(-15.6833, 13.6833, geotransform) # coordonnées en pixel de la station de Mako

strahlers = pcr.readmap('Data/Streams/strahler.map')
kedougou_strahler = strahler_value_point(strahlers, kedoudou_x_in_pixel, kedougou_y_in_pixel)
mako_strahler = strahler_value_point(strahlers, mako_x_in_pixel, mako_y_in_pixel)
kedougou_strahler

1

In [35]:
def visualization_ldd_and_strahlers(ldd_numpy, strahler_calibrated, strahlers_complete, kedougou_coords, mako_coords, threshold):
    
    fig, (ax1, ax1, ax3) = plt.supbplot(2, 2, figsize=(20, 10))
    
    # Premiere carte: Une carte pour représenter le LDD
    im1 = ax1.imshow(ldd_array, cmap='viridis', origin='upper')
    ax1.plot([kedougou_coords[0], mako_coords[0]], [kedougou_coords[1], mako_coords[1]], color='red',
        s=100, label='Stations')
    ax1.set_title('Le Local Drain Direction', fonsize=16)
    ax1.legend()
    fig.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04, label='Local Drain Direction')
    
    # Deuxième carte: Tous les ordres de Strahlers
    im2 = ax2.imshow(strahlers_complete, cmap='viridis', origin='upper')
    ax2.plot([kedougou_coords[0], mako_coords[0]], [kedougou_coords[1], mako_coords[1]], color='red',
        s=100, label='Stations')
    ax2.set_title('Tous les ordres de stahlers', fonsize=16)
    ax2.legend()
    fig.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04, label='Ordres de Strahlers')
    
    # Troisième carte: Ordres seuil
    im3 = ax3.imshow(strahler_calibrated, cmap='viridis', origin='upper')
    ax3.plot([kedougou_coords[0], mako_coords[0]], [kedougou_coords[1], mako_coords[1]], color='red',
        s=100, label='Stations')
    ax3.set_title(f'Ordre de Strahler >= {threshold}', fonsize=16)
    ax3.legend()
    fig.colorbar(im3, ax=ax3, fraction=0.046, pad=0.04, label=f'Ordres >= {threshold}')
    
    # Titre général
    plt.subtitle('Validation graphique du réseau hydrographique', fontsize=20)
    plt.show()

In [15]:
# Visualisation
ldd_numpy = pcr_to_numpy(ldd) # convertir le LDD en tableau numpy
orders = stream_delineation(ldd) # Ordre de strahler, ordre maximum 11
orders_numpy = pcr_to_numpy(orders)
strahler_order_threshold = 8 # doit être calibré, ne peut pas être fait automatiquement
strahler_order_8 = orders >= 8 # Capturer tous les ordres >= 8
strahler_order_8_numpy = pcr_to_numpy(strahler_order_8_numpy)
geotransform = gdal.Open('Data/Aligned_rasters/mnt_aligned.tif').GetGeoTransform()
kedougou = geoToPixel(-12.1833, 12.55, geotransform) # Coordonnées en pixel de la station de Kédougou
mako = geoToPixel(-12.35, 12.8667, geotransform) # Coordonnée en pixel de la station de Mako
visualization_ldd_and_strahlers(ldd_numpy=ldd_numpy, 
                                strahler_calibrated=strahler_order_8_numpy, 
                                strahlers_complete=orders, 
                                kedougou_coords=kedougou, 
                                mako_coords=mako, 
                                threshold=8)

In [13]:
order_max = pcr.mapmaximum(strahlers)
order_max_value = pcr.cellvalue(order_max, 1, 1)[0]
for order in range(order_max, 0, -1):
    mask = pcr.ifthen(pcr.scalar(strahlers) == order, pcr.boolean(1))
    

11

In [7]:
def pcrasterNumpy(dataset):
    pcr.report(dataset, 'tmp.map')
    dataset = pcr.readmap('tmp.map')
    rows = pcr.clone().nrRows()
    cols = pcr.clone().nrCols()
    dataset_array = np.zeros((rows, cols))
    for row in range(rows):
        for col in range(cols):
            dataset_array[row, col] = pcr.cellvalue(dataset, row + 1, col + 1)[0]
    return dataset_array
#array = pcrasterNumpy(ldd)
#array

In [19]:
values, counts = np.unique(array, return_counts=True)
values, counts

(array([0., 1.]), array([19821135,    12777]))

In [8]:
ordre_11 = strahlers == 8
array = pcrasterNumpy(ordre_11)

In [12]:

plt.imshow(array, cmap='viridis', alpha=0.9, origin="upper")
plt.scatter(kedoudou_x_in_pixel, kedougou_y_in_pixel, c='red', label='Kedougou sur l"ordre 11', marker='x')
plt.scatter(mako_x_in_pixel, mako_y_in_pixel, c='red', label='Mako sur l"ordre 8', marker='*')
plt.plot([kedoudou_x_in_pixel, mako_x_in_pixel], [kedougou_y_in_pixel, mako_y_in_pixel], color='black',
        linestyle='--', label='Ligne Kédougou-Mako')
plt.legend()
plt.title("Validation visuelle avec Kédougou et Mako")
plt.show()

MemoryError: Unable to allocate 151. MiB for an array with shape (3717, 5336) and data type float64

<Figure size 640x480 with 1 Axes>

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
kedougou_strahler = strahler_value_point(strahlers, col, row)
kedougou_strahler

NameError: name 'col' is not defined

In [ ]:
#pcr.lddcreatedem('Data/PCRaster_files/mnt.map', 1e31, 1e31, 1e31, 1e31)